In [1]:
import duckdb

In [2]:
conn = duckdb.connect("apache_web_logs")

In [3]:
conn.execute("SHOW TABLES").fetchdf()

,name
0,joined_log_table


In [9]:
conn.execute("DESCRIBE joined_log_table").fetchdf()

,column_name,column_type,null,key,default,extra
0,host,VARCHAR,YES,NaN,NaN,NaN
1,datetime,TIMESTAMP,YES,NaN,NaN,NaN
2,method,VARCHAR,YES,NaN,NaN,NaN
3,request,VARCHAR,YES,NaN,NaN,NaN
4,status,VARCHAR,YES,NaN,NaN,NaN
5,bytes,BIGINT,YES,NaN,NaN,NaN
6,hostname,VARCHAR,YES,NaN,NaN,NaN


In [8]:
conn.execute("SELECT COUNT(*) FROM joined_log_table").fetchone()

(1899999,)

In [13]:
conn.execute("SELECT SUM(bytes) FROM joined_log_table").fetchone()[0]/1000/1000

22868.541936

In [23]:
conn.execute("SELECT status, (SUM(bytes)/1000/1000)MB FROM joined_log_table GROUP BY status HAVING MB > 0 ORDER BY MB ").fetchdf()

,status,MB
0,status=301,1.0
1,status=500,7.0
2,status=404,299.0
3,status=200,22560.0


In [24]:
conn.execute("SELECT method, (SUM(bytes)/1000/1000)MB FROM joined_log_table GROUP BY method HAVING MB > 0 ORDER BY MB").fetchdf()

,method,MB
0,POST,95.0
1,GET,22773.0


In [8]:
conn.execute("""
    SELECT 
        strftime(datetime, '%H')log_hour, 
        (SUM(bytes)/1000/1000)MB 
    FROM joined_log_table 
    GROUP BY log_hour 
    ORDER BY log_hour
    """
).fetchdf()

,log_hour,MB
0,03,23.0
1,04,304.0
2,05,322.0
3,06,360.0
4,07,641.0
5,08,1113.0
6,09,1599.0
7,10,1833.0
8,11,2246.0
9,12,1787.0


In [37]:
conn.execute("""
    WITH table_1 AS (
        SELECT 
            strftime(datetime, '%H')dt,
            hostname,
            bytes
        FROM joined_log_table
    ),
    table_2 AS (SELECT 
        dt,
        hostname,
        bytes,
        ROW_NUMBER() OVER (PARTITION BY dt ORDER BY bytes DESC) row_number
    FROM table_1)
    SELECT dt, hostname, (bytes/1000)KB FROM table_2 WHERE row_number = 1
    ORDER BY 1
    """
).fetchdf()

,dt,hostname,KB
0,03,95.81.118.56,185
1,04,46.4.203.149,584
2,05,172.20.2.174,912
3,06,159.65.185.106,769
4,07,crawl-66-249-66-92.googlebot.com,872
5,08,40.77.167.13,1249
6,09,5.121.8.89,708
7,10,5.160.116.166,708
8,11,207.46.13.199,1147
9,12,5.219.82.41,776


In [ ]:
conn.execute("""
    SELECT 
        strftime(datetime, '%H')dt,
        AVG(bytes) OVER five as bytes_5_hours_mov_avg
    FROM joined_log_table 
    WINDOW
        five AS (
            ORDER BY datetime
            RANGE BETWEEN INTERVAL 2 HOURS PRECEDING AND INTERVAL 2 HOURS FOLLOWING)
    ORDER BY 1, 2
    """
).fetchdf()